In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
# from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [3]:
Themes = pd.read_excel("Data.xlsx", sheet_name="Themes")
pd.set_option('display.max_columns', None)
Themes.head()

,Unnamed: 0,Возраст,Пол,ОУ,Факультет_ГЭУ,Курс,Профиль,Курс_1,МП,Раб_статус,Старт карьеры,Инвестиции beginner,Инвестиции intermediate,Стартапы,Креативные индустрии,Импортозамещение в технологиях,Санкции и возможности,IT,Soft skills,Профессии будущего,Психология и саморазвитие,Фин.грамотность,Социальное предпринимательство,Мировая экономика,География карьеры,СКО,среднее,Старт карьеры норм.,Инвестиции beginner норм.,Инвестиции intermediate норм.,Стартапы норм.,Креативные индустрии норм.,Импортозамещение в технологиях норм.,Санкции и возможности норм.,IT норм.,Soft skills норм.,Профессии будущего норм.,Психология и саморазвитие норм.,Фин.грамотность норм.,Социальное предпринимательство норм.,Мировая экономика норм.,География карьеры норм.
0,0,20,Женский,СПбГЭУ,Факультет управления,1,NaN,NaN,Санкт-Петербург и Ленинградская область,"Нет, не имею опыта работы и стажировок",10,7,7,2,9,2,9,8,10,8,9,9,6,8,9,3.309438,6.571429,0.326338,0.228436,0.228436,0.065268,0.293704,0.065268,0.293704,0.261070,0.326338,0.261070,0.293704,0.293704,0.195803,0.261070,0.293704
1,1,19,Женский,СПбГЭУ,Гуманитарный факультет,1,NaN,NaN,Другой регион России,"Нет, не имею опыта работы и стажировок",10,10,9,8,7,6,7,8,9,10,10,10,10,9,8,1.573592,8.142857,0.292478,0.292478,0.263230,0.233982,0.204734,0.175487,0.204734,0.233982,0.263230,0.292478,0.292478,0.292478,0.292478,0.263230,0.233982
2,2,20,Женский,СПбГЭУ,Факультет информатики и прикладной математики,3,NaN,NaN,Санкт-Петербург и Ленинградская область,"Да, работаю / прохожу стажировку по профилю об...",3,10,7,4,1,1,10,10,5,7,8,10,10,10,6,3.891382,5.142857,0.102899,0.342997,0.240098,0.137199,0.034300,0.034300,0.342997,0.342997,0.171499,0.240098,0.274398,0.342997,0.342997,0.342997,0.205798
3,3,20,Женский,СПбГЭУ,"Факультет сервиса, туризма и гостеприимства",3,NaN,NaN,Санкт-Петербург и Ленинградская область,"Нет, не имею опыта работы и стажировок",10,2,2,5,8,7,8,3,10,7,10,6,4,4,2,3.109126,6.000000,0.395285,0.079057,0.079057,0.197642,0.316228,0.276699,0.316228,0.118585,0.395285,0.276699,0.395285,0.237171,0.158114,0.158114,0.079057
4,4,18,Женский,СПбГЭУ,Гуманитарный факультет,1,NaN,NaN,Другой регион России,"Да, работаю не по профилю обучения",10,8,5,8,8,10,9,8,7,9,10,5,8,8,5,1.704336,8.285714,0.321081,0.256865,0.160540,0.256865,0.256865,0.321081,0.288973,0.256865,0.224756,0.288973,0.321081,0.160540,0.256865,0.256865,0.160540


In [5]:
Themes = pd.read_excel("Data.xlsx", sheet_name="Themes")

Themes = Themes.drop(['Unnamed: 0', 'Старт карьеры', 'Инвестиции beginner', 'Инвестиции intermediate', 'Стартапы', 'Креативные индустрии', 'Импортозамещение в технологиях', 'Санкции и возможности', 'IT', 'Soft skills', 'Профессии будущего', 'Психология и саморазвитие', 'Фин.грамотность', 'Социальное предпринимательство', 'Мировая экономика', 'География карьеры'], axis=1)
Themes.rename(columns = {'Старт карьеры норм.': 'Старт карьеры', 'Инвестиции beginner норм.': 'Инвестиции beginner', 'Инвестиции intermediate норм.': 'Инвестиции intermediate', 'Стартапы норм.': 'Стартапы', 'Креативные индустрии норм.': 'Креативные индустрии норм.', 'Импортозамещение в технологиях норм.': 'Импортозамещение в технологиях', 'Санкции и возможности норм.': 'Санкции и возможности', 'IT норм.': 'IT', 'Soft skills норм.': 'Soft skills', 'Профессии будущего норм.': 'Профессии будущего', 'Психология и саморазвитие норм.': 'Психология и саморазвитие', 'Фин.грамотность норм.': 'Фин.грамотность', 'Социальное предпринимательство норм.': 'Социальное предпринимательство', 'Мировая экономика норм.': 'Мировая экономика', 'География карьеры норм.': 'География карьеры'}, inplace = True)

Themes = pd.concat((Themes,pd.get_dummies(Themes.Пол)),1)
Themes = pd.concat((Themes,pd.get_dummies(Themes.ОУ)),1)
Themes['выбор_1'] = Themes.iloc[:, 9:15].T.apply(lambda x: x.nlargest(1).idxmin())
Themes['выбор_2'] = Themes.iloc[:, 9:15].T.apply(lambda x: x.nlargest(3).idxmin())

def rule(x, y):
    if x == y:
         return x
    else:
         return y

Themes['выбор'] = Themes.apply(lambda x: rule(x['выбор_1'], x['выбор_2']), axis =  1)

Themes_new = Themes[['Возраст', 'Женский', 'Мужской', 'Другой вуз', 'Закончил обучение', 'Колледж / техникум / профессиональное(ый) училище или лицей', 'СПбГЭУ', 'Школа / гимназия / лицей', 'выбор']]

/var/folders/9z/l1j6fqx94236qbp68r3y8_jh0000gn/T/ipykernel_19882/3552971177.py:6: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  Themes = pd.concat((Themes,pd.get_dummies(Themes.Пол)),1)
/var/folders/9z/l1j6fqx94236qbp68r3y8_jh0000gn/T/ipykernel_19882/3552971177.py:7: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  Themes = pd.concat((Themes,pd.get_dummies(Themes.ОУ)),1)


In [6]:
y = Themes_new.выбор
x = Themes_new.drop('выбор',
            axis=1)  # все кроме последней колонки (axis=1 => по столбцам)

model = LogisticRegression()
model.fit(x, y)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [7]:
Themes_new.head()

,Возраст,Женский,Мужской,Другой вуз,Закончил обучение,Колледж / техникум / профессиональное(ый) училище или лицей,СПбГЭУ,Школа / гимназия / лицей,выбор
0,20,1,0,0,0,0,1,0,Старт карьеры
1,19,1,0,0,0,0,1,0,Старт карьеры
2,20,1,0,0,0,0,1,0,Инвестиции beginner
3,20,1,0,0,0,0,1,0,Старт карьеры
4,18,1,0,0,0,0,1,0,Старт карьеры


In [8]:
{col:[0] for col in x.columns}

{'Возраст': [0],
 'Женский': [0],
 'Мужской': [0],
 'Другой вуз': [0],
 'Закончил обучение': [0],
 'Колледж / техникум / профессиональное(ый) училище или лицей': [0],
 'СПбГЭУ': [0],
 'Школа / гимназия / лицей': [0]}

In [9]:
example = {'Возраст': [21],
 'Женский': [1],
 'Мужской': [0],
 'Другой вуз': [0],
 'Закончил обучение': [0],
 'Колледж / техникум / профессиональное(ый) училище или лицей': [0],
 'СПбГЭУ': [1],
 'Школа / гимназия / лицей': [0]}

example_df = pd.DataFrame(example)

result = model.predict(example_df)
print(result)

['Старт карьеры']


In [10]:
example = {'Возраст': [48],
 'Женский': [0],
 'Мужской': [1],
 'Другой вуз': [0],
 'Закончил обучение': [1],
 'Колледж / техникум / профессиональное(ый) училище или лицей': [0],
 'СПбГЭУ': [0],
 'Школа / гимназия / лицей': [0]}

example_df = pd.DataFrame(example)

result = model.predict(example_df)
print(result)

['Стартапы']
